#### KNN

In [2]:
# pip install scikit-surprise

In [3]:
import ast
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [4]:
# 과거 데이터 로드
past_data = pd.read_csv('../train_data/Data4_train_nan.csv')

In [5]:
past_data

,musical_id,title,poster_url,genre,date,start_date,end_date,location,actors,age_rating,...,describe,synopsis,synopsis_clear,tokenized_data,synopsis_vector,synopsis_numpy,synopsis_numpy_scale,tag1,tag2,tag3
0,3885,실연남녀,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20071013 20071209,2007-10-01,2007-10-13,대학로 티오엠 1관,엄기준신성록강지우양소민한애리변우민성민,7세이상,...,뮤지컬 실연남녀 는 브라운관과 뮤지컬계를 넘나드는 최고의 배우들의 출연진으로도 화제...,어리버리 조폭 형제가 경영하는 아트 퓨쳐산장 이들은 6개월 전 아트 퓨쳐산장에 사채...,조폭 형제 경영 아트 퓨쳐 산장 이 개월 전 아트 퓨쳐 산장 사채 빚 갑자기 심장마...,"['조폭', '형제', '경영', '아트', '퓨쳐', '산장', '이', '개월'...",b'\x0e\x11u<\xdeM1=\xa1kX=\xd11\x06\xbe?\x03\x...,[ 0.01495768 0.04328715 0.05283702 -0.131049...,"[-0.6652450561523438, 0.25175362825393677, -0....",사랑,사람,슬픔
1,3635,김종욱 찾기,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20061212 20070408,2006-12-01,2006-12-12,JTN 아트홀 1관,신성록전병욱오나라안유진원종환김재범김지현,만 13세이상,...,No1 창작뮤지컬의 신화뮤지컬 김종욱찾기연출김지연는 창작뮤지컬계 블루칩 장유정극작작...,첫사랑을 찾아나선 여자와첫사랑을 찾아주는 남자의 티격태격 로맨틱 코미디운명은 항상 ...,첫사랑 나선 여자 첫사랑 남자 티격태격 로맨틱 코미디 운명 항상 곁 있다 단지 뿐 ...,"['첫사랑', '나선', '여자', '첫사랑', '남자', '티격태격', '로맨틱'...",b'\xddC\xf5=I@Z=%\xfd\x8b\xbc\xd8\x03==\xd6q\x...,[ 0.11975835 0.05328396 -0.01708848 0.046146...,"[0.05565793439745903, 0.32731708884239197, -0....",사랑,오페라,NaN
2,4829,첫사랑,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070327 20070617,2007-03-01,2007-03-27,대학로 티오엠 1관,조정석홍광호해이전경수이정섭김성기김인수,만 6세이상,...,뮤지컬 첫사랑은 어느 바닷가를 배경으로 이루어지지 못한 첫사랑에 대한 아련한 추억을...,시간을 조금만 되돌릴 수 있다면어릴 적부터 한 동네에서 자란 해수와 선이는 오누이 ...,시간 조금 수 있다 적 동네 해수 오누이 같다 연인 해수 아버지 박씨 운영 까페 마...,"['시간', '조금', '수', '있다', '적', '동네', '해수', '오누이'...","b""S\xa2\xa0<\xeb\xad\xd5<\xd6\x93\xbe\xbd\x01\...",[ 0.01960865 0.02608391 -0.09305541 -0.089630...,"[-0.6332519054412842, 0.12171854823827744, -1....",오페라,NaN,NaN
3,3322,미라클,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070601 20130217,2007-06-01,2007-06-01,미라클 씨어터 1관,신현묵조호균김고운원혜지남궁인윤도훈김학규,만 12세이상,...,미라클 종합병원에서 벌어지는 웃음과 감동의 뮤지컬2003년 초연 이래로 16번의 앵...,인기그룹 핫바의 멤버인 가수 희동은 어느 날 교통사고로 식물인간이 된다 하지만 희동...,인기 그룹 핫바 멤버 가수 희다 은 날 교통사고 식물인간 희다 동의 영혼 몸밖 병실...,"['인기', '그룹', '핫바', '멤버', '가수', '희다', '은', '날',...",b'^\xb5D>\xb63\xad=P\x8e >.\x97\x02\xbe\xee+\x...,[ 1.92098111e-01 8.45712870e-02 1.56792879e-...,"[0.5532688498497009, 0.5638102889060974, 0.553...",오페라,NaN,NaN
4,4424,대장금,httpticketimageinterparkcomPlayDictionaryDATAP...,창작,20070526 20070617,2007-05-01,2007-05-26,예술의전당 오페라극장,최보영안유진김우형김소현이태원류수화손광업한애리한성식김나윤김일우최윤정장효상,만 7세이상,...,3색의 무대로 계승하는 국민드라마 대장금의 신화같은 이야기 다른 감동의 무대지난 2...,어머니의 유언에 따르기 위해 궁으로 들어가 생각시가 된어린 장금은 이것 저것 묻기 ...,어머니 유언 위해 궁 생각 시가 장금 것 것 좋아하다 아이 궁 안 성장하다 뛰어나다...,"['어머니', '유언', '위해', '궁', '생각', '시가', '장금', '것'...",b'@\x8a\xa8=\x8e\xd7\xe0=\x1e\x10\x9b=N%\xf6\x...,[ 0.08229494 0.10978614 0.07571433 -0.120188...,"[-0.20204538106918335, 0.7544031143188477, -0....",아이,사랑,오페라
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9046,150211,브레멘 음악대 부산,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20191116 20191117,2019-11-01,2019-11-16,신세계센텀시티 문화홀9층,NaN,24개월이상,...,NaN,집에서 키워지던 동물들이 주인의 천대를 받고쫓겨나거나 스스로의 자유를 위해 집을 나...,집 동물 주인 천대 스스로 자유 위해 집 친구 멋지다 악기 음악회 만들기 즐겁다 시...,"['집', '동물', '주인', '천대', '스스로', '자유', '위해', '집'...","b'e#^\xbc\xc5o\x10\xbd\x07,\x02>\xa4\xd4\x1c>\...",[-1.35582434e-02 -3.52628417e-02 1.27121076e-...,"[-0.8614004254341125, -0.34198644757270813, 0....",친구,예술,NaN
9047,150242,헤이지니 럭키강이 2 청주,httpticketimageinterparkcomPlayDictionaryDATAP...,어린이가족,20191228 20191229,2019-12-01,2019-12-28,청주예술의전당 대공연장,NaN,24개월이상,...,NaN,딩동 마법의 세계로 가야할 택배상자가 지니강이 집으로 배달되었다신비로운 마법의 열쇠...,딩동 마법 세계 가야 택배 상자 지니 강 집 배달 신비롭다 마법 열쇠 지니 강 비밀...,"['딩동', '마법', '세계', '가야', '택배', '상자', '지니', '강'...","b""\xe1uF=`\x1c\xb3<\x01\x92\x93>\xfc&~\xbe|\xa...",[ 0.04845226 0.02186412 0.2882233 -0.248195...,"[-0.4348424971103668, 0.0898221805691719, 1.50...",마법,오페라,NaN
9048,150349,난타 대구,httpticketimageinterparkcomPlayDictionaryDATAP...,퍼포먼스,20191220 20191225,2019-12-01,2019-12-20,대구 수성아트피아 용지홀,NaN,36개월이상,...,NaN,세 명의 요리사가 하루 일과를 시작한다즐겁게 요리를 준비하는 동안 지배인의 깜짝

In [6]:
# 데이터프레임에서 synopsis_numpy_scale 열의 값을 파싱하여 리스트로 변환
past_data['synopsis_numpy_scale'] = past_data['synopsis_numpy_scale'].apply(ast.literal_eval)

In [7]:
# StandardScaler를 사용하여 특성들을 표준 스케일링
scaler_past = StandardScaler()
past_data_scaled = scaler_past.fit_transform(np.vstack(past_data['synopsis_numpy_scale']))

In [8]:
past_data_scaled

array([[-0.66524505,  0.25175363, -0.19866295, ...,  0.20609155,
        -0.01210716, -1.45184948],
       [ 0.05565794,  0.32731709, -0.70489442, ..., -0.68315441,
         0.11303943,  1.58590651],
       [-0.6332519 ,  0.12171855, -1.25486338, ...,  0.09821291,
        -0.01894334,  0.43942407],
       ...,
       [ 0.31357838,  1.67281711,  0.63475806, ...,  0.1058756 ,
        -0.0190074 , -0.81816704],
       [ 2.09098461, -1.6402173 , -1.67033899, ..., -1.13997639,
         0.78284753,  1.45906688],
       [-0.61023718,  0.01180978, -0.91649306, ..., -0.00324211,
         0.23349878,  0.26390224]])

In [9]:
# KNN 모델 초기화
knn_model_past = NearestNeighbors(n_neighbors=7, metric='euclidean')
knn_model_past.fit(past_data_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=7)

In [10]:
# 현재 데이터 로드
present_data = pd.read_csv('../present_data/Data4_present_nan.csv')

In [11]:
# 데이터프레임에서 synopsis_numpy_scale 열의 값을 파싱하여 리스트로 변환
present_data['synopsis_numpy_scale'] = present_data['synopsis_numpy_scale'].apply(ast.literal_eval)

In [12]:
# StandardScaler를 사용하여 특성들을 표준 스케일링
scaler_present = StandardScaler()
present_data_scaled = scaler_present.fit_transform(np.vstack(present_data['synopsis_numpy_scale']))

In [13]:
# KNN 모델 초기화
knn_model_present = NearestNeighbors(n_neighbors=6, metric='euclidean')
knn_model_present.fit(present_data_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=6)

In [17]:
# 과거 데이터에서 작품을 선택 (예: musical_id가 3885인 작품)
selected_work_index_past = past_data[past_data['musical_id'] == 3885].index[0]

In [18]:
# 선택한 작품과 유사한 작품 찾기
distances, indices = knn_model_present.kneighbors([present_data_scaled[selected_work_index_past]]) # 내적으로 유클리디안 거리가 계산됨

In [19]:
# 최대 거리와 최소 거리 계산
max_distance = distances.max()
min_distance = distances.min()

# 정규화된 유사도 계산
normalized_distances = (1 - (distances - min_distance) / (max_distance - min_distance))

# 유사한 작품 출력
print("선택한 작품:")
print(past_data.iloc[selected_work_index_past]['title'])
print("")
print("유사한 작품들:")
for i in range(1, len(indices.flatten())):
    index = indices.flatten()[i]
    normalized_distance = normalized_distances.flatten()[i]
    title = present_data.iloc[index]['title']
    musical_id = present_data.iloc[index]['musical_id']
    print(f"유사한 작품: {title}, Musical ID: {musical_id}, 정규화된 유사도: {normalized_distance}")

선택한 작품:
실연남녀

유사한 작품들:
유사한 작품: 레미제라블, Musical ID: 190722, 정규화된 유사도: 1.0
유사한 작품: 옹알스, Musical ID: 192343, 정규화된 유사도: 0.1289757565343258
유사한 작품: 오페라의 유령, Musical ID: 183052, 정규화된 유사도: 0.07012744187378717
유사한 작품: 그림책 속 제주이야기, Musical ID: 190256, 정규화된 유사도: 0.06582867629190003
유사한 작품: 왓 이프, Musical ID: 186831, 정규화된 유사도: 0.0
